In [3]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd

def modify_parameters(parameters):
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=1)
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=2)
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=3)
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[3].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=4)
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=5)
    parameters.gov.contrib.ubi_center.flat_tax.abolish_federal_income_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.abolish_payroll_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.abolish_self_emp_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.deduct_ptc.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.contrib.ubi_center.flat_tax.rate.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=0.29)
    parameters.gov.ssa.ssi.abolish_ssi.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.usda.snap.abolish_snap.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.gov.usda.snap.emergency_allotment.allowed.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=False)
    parameters.gov.usda.wic.abolish_wic.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    parameters.simulation.reported_state_income_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    return parameters

class bs_reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)


def baseline_modify_parameters(parameters):
    parameters.simulation.reported_state_income_tax.update(start=instant("2023-01-01"), stop=instant("2032-12-31"), value=True)
    return parameters

class baseline_reform(Reform):
    def apply(self):
        self.modify_parameters(baseline_modify_parameters)


#baseline = Microsimulation(reform=baseline_reform, dataset="enhanced_cps_2023")
reformed = Microsimulation(reform=bs_reform, dataset="enhanced_cps_2023")

In [30]:
sim = Microsimulation(dataset="enhanced_cps_2023")
sim.trace = True
sim.calculate("is_ssi_disabled", 2023).sum()/1e6
#sim.tracer.print_computation_log()

35.37348741684565

In [4]:
age = reformed.calculate("age", 2024)
disabled = reformed.calculate("is_ssi_disabled", 2024)

print(f"Young children: {(age < 6).sum()/1e6:.1f}m")
print(f"Older children: {((age >= 6) * (age < 18)).sum()/1e6:.1f}m")
print(f"Adults: {((age >= 18) * (age < 65)).sum()/1e6:.1f}m")
print(f"Seniors: {(age >= 65).sum()/1e6:.1f}m")
print(f"SSI disabled: {(disabled).sum()/1e6:.1f}m")

Young children: 22.1m
Older children: 54.5m
Adults: 194.2m
Seniors: 64.0m
SSI disabled: 3.7m


In [9]:
from policyengine_us.data import RawCPS_2022

cps = RawCPS_2022().load()

Downloaded ASEC: 100%|██████████| 150M/150M [00:40<00:00, 3.68MiB/s]    


In [13]:
DISABILITY_FLAGS = [
        "PEDIS" + i for i in ["DRS", "EAR", "EYE", "OUT", "PHY", "REM"]
    ]

person = cps["person"]

In [17]:
(person["A_FNLWGT"] * (person[DISABILITY_FLAGS] == 1).any(axis=1)).sum()/1e6/100

33.28557379

In [18]:
from policyengine_us.data import CPS_2023, EnhancedCPS_2023

is_ssi_disabled = EnhancedCPS_2023().load("is_ssi_disabled")

In [20]:
weight = reformed.calculate("person_weight", 2024)

In [23]:
(weight.values * is_ssi_disabled).sum() / 1e6

35.373484

In [25]:
reformed.calculate("is_ssi_disabled", 2024).sum()/1e6

3.7319723368099926

In [20]:
enhanced_cps = EnhancedCPS_2023().load_dataset()

In [23]:
enhanced_cpsb

{'age': array([66, 68, 52, ..., 59, 60, 18]),
 'alimony_income': array([0, 0, 0, ..., 0, 0, 0]),
 'child_support_expense': array([0, 0, 0, ..., 0, 0, 0]),
 'child_support_received': array([0, 0, 0, ..., 0, 0, 0]),
 'childcare_expenses': array([0, 0, 0, ..., 0, 0, 0]),
 'county_fips': array([0, 0, 0, ..., 3, 3, 3]),
 'cps_race': array([ 1,  1,  1, ..., 15,  4, 15]),
 'disability_benefits': array([0, 0, 0, ..., 0, 0, 0]),
 'employment_income': array([      0.        ,       0.        ,       0.        , ...,
        2470576.25738139, 5839543.88108328,  513879.86153533]),
 'family_id': array([3.10000000e+01, 4.10000000e+01, 1.51000000e+02, ...,
        1.00889761e+08, 1.00889771e+08, 1.00889781e+08]),
 'farm_income': array([0., 0., 0., ..., 0., 0., 0.]),
 'farm_rent_income': array([0., 0., 0., ..., 0., 0., 0.]),
 'free_school_meals_reported': array([  0,   0,   0, ..., 952, 952, 476]),
 'has_marketplace_health_coverage': array([False, False, False, ..., False, False, False]),
 'health_ins

In [24]:
import numpy as np

enhanced_cps["is_ssi_disabled"] = np.array(list(is_ssi_disabled) * 2)

In [25]:
EnhancedCPS_2023().save_dataset(enhanced_cps)

In [6]:
numbers = list(range(1, 6))
ubi_groups = [
    "Young child",
    "Older child",
    "Young adult",
    "Older adult",
    "Senior",
    "Disabled",
]

for i, group in zip(numbers, ubi_groups):
    print(f"{group}: {(ubi == i).sum()/1e6:,.0f}")

Young child: $0
Older child: $1
Young adult: $15
Older adult: $58
Senior: $34


In [ ]:
ubi = reform